In [3]:
from pint import UnitRegistry
from functools import wraps

units = UnitRegistry()
Q_ = units.Quantity

# pint에 사용자 정의 단위 등록 (먼저 정의!)
units.define('USD = [currency_USD]')
units.define('KRW = [currency_KRW]')

# 통화 단위 정의 (USD, KRW) (정의 후에 units.USD, units.KRW 로 접근)
USD = units.USD
KRW = units.KRW


exchange_rate_usd_to_krw = Q_(1440, KRW/USD)


def ensure_currency_unit(expected_unit): # 간단한 단위 검증 데코레이터
    """입력 금액이 특정 통화 단위인지 검증하는 데코레이터 (간단 버전)"""
    def decorator_ensure_currency_unit(func):
        @wraps(func)
        def wrapper_ensure_currency_unit(amount, *args, **kwargs): # 첫 번째 인자만 단위 검증 (간단화)
            if not isinstance(amount, Q_):
                raise ValueError("입력 금액은 pint Quantity 객체여야 합니다.")
            if amount.units != expected_unit: # 단위 일치 여부 확인 (엄격한 비교)
                raise ValueError(f"입력 금액 단위 오류: 예상 단위는 '{expected_unit}' 입니다. 입력된 단위: '{amount.units}'")
            return func(amount, *args, **kwargs) # 원래 함수 호출
        return wrapper_ensure_currency_unit
    return decorator_ensure_currency_unit


@ensure_currency_unit(USD) # USD 단위 검증 데코레이터 적용
def calculate_exchange_amount_decorated(amount, exchange_rate):
    """단위 인식 데코레이터가 적용된 환율 계산 함수 (USD 입력 검증)"""
    return amount * exchange_rate


# 예시: 100 USD 를 KRW 로 환산 (데코레이터 적용 함수 호출)
usd_amount_decorated = Q_(100, USD)
krw_amount_decorated = calculate_exchange_amount_decorated(usd_amount_decorated, exchange_rate_usd_to_krw)
print(f"{usd_amount_decorated} 는 {krw_amount_decorated} (데코레이터 적용) 입니다.")


# 단위 오류 예시 (주석 해제 후 실행 시 오류 발생)
# krw_amount_wrong_unit = Q_(100000, KRW) # KRW 단위 금액
# calculate_exchange_amount_decorated(krw_amount_wrong_unit, exchange_rate_usd_to_krw) # 오류 발생: USD 단위가 아님

100 USD 는 144000 KRW (데코레이터 적용) 입니다.
